## Heat Maps

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.stats import linregress
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

import gmaps
import pandas as pd
import requests
import json

# Google developer API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

filepath = "Lane_cities.csv"
df = pd.read_csv(filepath)

df.head()

,City,Country,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness
0,Busselton,AU,-33.65,115.33,57.61,98,3.15,38
1,Misratah,LY,32.38,15.09,69.39,65,10.96,1
2,Qaanaaq,GL,77.48,-69.36,-12.33,70,8.90,0
3,Hobart,AU,-42.88,147.33,58.15,54,5.82,20
4,Butaritari,KI,3.07,172.79,81.34,72,11.63,92


In [2]:
 # Store latitude and longitude in locations
locations = df[["Latitude", "Longitude"]]

humidity = df["Humidity"]

In [3]:
# Plot Heatmap
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=750)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [4]:
mask = (df["Temperature"]<=85) & (df["Temperature"] >=75) & (df["Wind Speed"] <=5) & (df["Cloudiness"] == 0)
idealWeather = df.loc[mask].reset_index(drop=True)
idealWeather

,City,Country,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness
0,Gunjur,GM,13.20,-16.73,80.60,83,3.36,0
1,Santarém,BR,-2.44,-54.71,82.40,83,4.70,0
2,Gogrial,SS,8.53,28.10,76.59,41,0.98,0
3,Chimoio,MZ,-19.12,33.48,75.09,69,4.70,0
4,Moreira Sales,BR,-24.06,-53.01,75.92,43,0.74,0
5,Mumbai,IN,19.01,72.85,78.80,54,4.70,0
6,Kang,BW,-23.68,22.79,81.75,17,4.27,0


In [5]:
#inits
names = []

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in idealWeather.iterrows():
    
    location = f"{row.Latitude}, {row.Longitude}"

    # add keyword to params dict
    params['location'] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row.City}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest Hotel is {results[0]['name']}.")     
        name = results[0]['name']
        names.append(name)
    
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        names.append(np.nan)
        
    print("------------")
              
idealWeather["Hotel_Name"] = names

Retrieving Results for Index 0: Gunjur.
Closest Hotel is Sandally.
------------
Retrieving Results for Index 1: Santarém.
Closest Hotel is Brytto Design.
------------
Retrieving Results for Index 2: Gogrial.
Closest Hotel is Alek.
------------
Retrieving Results for Index 3: Chimoio.
Closest Hotel is Amado Mobílias Lda.
------------
Retrieving Results for Index 4: Moreira Sales.
Closest Hotel is Wallison Rodrigues.
------------
Retrieving Results for Index 5: Mumbai.
Missing field/result... skipping.
------------
Retrieving Results for Index 6: Kang.
Closest Hotel is Goo Baitshepi Inn.
------------


In [6]:
idealWeather

,City,Country,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness,Hotel_Name
0,Gunjur,GM,13.20,-16.73,80.60,83,3.36,0,Sandally
1,Santarém,BR,-2.44,-54.71,82.40,83,4.70,0,Brytto Design
2,Gogrial,SS,8.53,28.10,76.59,41,0.98,0,Alek
3,Chimoio,MZ,-19.12,33.48,75.09,69,4.70,0,Amado Mobílias Lda
4,Moreira Sales,BR,-24.06,-53.01,75.92,43,0.74,0,Wallison Rodrigues
5,Mumbai,IN,19.01,72.85,78.80,54,4.70,0,NaN
6,Kang,BW,-23.68,22.79,81.75,17,4.27,0,Goo Baitshepi Inn


In [7]:
list_info = []

for indx, row in idealWeather.iterrows():
    info = f"City: {row.City}<br>-----------<br>Country: {row['Country']}<br>-----------<br>Hotel Name: {row['Hotel_Name']}"
    list_info.append(info)

In [8]:
coords = df[["Latitude", "Longitude"]]
coords_ideal = idealWeather[["Latitude", "Longitude"]]

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coords_ideal, info_box_content=list_info)
# Add the layer to the map
fig.add_layer(markers)

heat = gmaps.heatmap_layer(coords, weights = humidity, dissipating=False, max_intensity=500)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…